In [ ]:
#https://towardsdatascience.com/an-end-to-end-project-on-time-series-analysis-and-forecasting-with-python-4835e6bf050b

In [14]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'


In [92]:
#raw_df = pd.read_csv("timeseries_vqa.csv")
raw_df = pd.read_csv("C:\\Users\\mcinn\\Documents\\timeseries_vqa.csv")
raw_df.head()

,BC Sales Channel,AWC and Comp,AWC Fiscal Year Month Abbr,Brand,Varietal Group,9L Cases,AWC FYr & Pd No,Avg Sales Price,Sales Dollars
0,BCLS,CONSTELLATION BRANDS,F2017 FEB,BLACK SAGE VINEYARD,CABERNET FRANC,17,201712,$28.74,"5,912"
1,BCLS,CONSTELLATION BRANDS,F2017 FEB,BLACK SAGE VINEYARD,CABERNET SAUVIGNON,1,201712,$28.74,368
2,BCLS,CONSTELLATION BRANDS,F2017 FEB,BLACK SAGE VINEYARD,MERLOT,11,201712,$28.74,"3,698"
3,BCLS,CONSTELLATION BRANDS,F2017 FEB,BLACK SAGE VINEYARD,OTHER,2,201712,$29.89,909
4,BCLS,CONSTELLATION BRANDS,F2017 FEB,CULMINA,CHARDONNAY,0,201712,$41.50,85


In [93]:
raw_df = raw_df.rename({'AWC Fiscal Year Month Abbr': 'Date'}, axis=1) 
raw_df['Date'] = raw_df['Date'].str[1:]
raw_df.head()

,BC Sales Channel,AWC and Comp,Date,Brand,Varietal Group,9L Cases,AWC FYr & Pd No,Avg Sales Price,Sales Dollars
0,BCLS,CONSTELLATION BRANDS,2017 FEB,BLACK SAGE VINEYARD,CABERNET FRANC,17,201712,$28.74,"5,912"
1,BCLS,CONSTELLATION BRANDS,2017 FEB,BLACK SAGE VINEYARD,CABERNET SAUVIGNON,1,201712,$28.74,368
2,BCLS,CONSTELLATION BRANDS,2017 FEB,BLACK SAGE VINEYARD,MERLOT,11,201712,$28.74,"3,698"
3,BCLS,CONSTELLATION BRANDS,2017 FEB,BLACK SAGE VINEYARD,OTHER,2,201712,$29.89,909
4,BCLS,CONSTELLATION BRANDS,2017 FEB,CULMINA,CHARDONNAY,0,201712,$41.50,85


In [94]:
import datetime
raw_df['Date'] = pd.to_datetime(raw_df['Date'], errors='coerce')
raw_df['Date'] = raw_df['Date'].dt.strftime('%D')
raw_df.head()

,BC Sales Channel,AWC and Comp,Date,Brand,Varietal Group,9L Cases,AWC FYr & Pd No,Avg Sales Price,Sales Dollars
0,BCLS,CONSTELLATION BRANDS,02/01/17,BLACK SAGE VINEYARD,CABERNET FRANC,17,201712,$28.74,"5,912"
1,BCLS,CONSTELLATION BRANDS,02/01/17,BLACK SAGE VINEYARD,CABERNET SAUVIGNON,1,201712,$28.74,368
2,BCLS,CONSTELLATION BRANDS,02/01/17,BLACK SAGE VINEYARD,MERLOT,11,201712,$28.74,"3,698"
3,BCLS,CONSTELLATION BRANDS,02/01/17,BLACK SAGE VINEYARD,OTHER,2,201712,$29.89,909
4,BCLS,CONSTELLATION BRANDS,02/01/17,CULMINA,CHARDONNAY,0,201712,$41.50,85


In [95]:
raw_df['Date'].min(), raw_df['Date'].max()


('01/01/18', '12/01/20')

In [96]:
raw_df['Varietal Group'].unique()

array(['CABERNET FRANC', 'CABERNET SAUVIGNON', 'MERLOT', 'OTHER',
       'CHARDONNAY', 'RED BLEND', 'RIESLING', 'VIDAL', 'PINOT NOIR',
       'CHENIN BLANC', 'PINOT BLANC', 'PINOT GRIGIO', 'SHIRAZ',
       'ZINFANDEL', 'WHITE BLEND', 'SAUVIGNON BLANC', 'VIOGNIER', 'ROSE',
       'GEWURZTRAMINER', 'TEMPRANILLO', 'MALBEC', 'UNASSIGNED'],
      dtype=object)

In [97]:
cabernet_franc = raw_df.loc[raw_df['Varietal Group'] == 'CABERNET FRANC']
cabernet_sav = raw_df.loc[raw_df['Varietal Group'] == 'CABERNET SAUVIGNON']
merlot = raw_df.loc[raw_df['Varietal Group'] == 'MERLOT']
chardonnay = raw_df.loc[raw_df['Varietal Group'] == 'CHARDONNAY']
red_blend = raw_df.loc[raw_df['Varietal Group'] == 'RED BLEND']
riesling = raw_df.loc[raw_df['Varietal Group'] == 'RIESLING']
vidal = raw_df.loc[raw_df['Varietal Group'] == 'VIDAL']
pinot_noir = raw_df.loc[raw_df['Varietal Group'] == 'PINOT NOIR']
chenin_blanc = raw_df.loc[raw_df['Varietal Group'] == 'CHENIN BLANC']
pinot_blanc = raw_df.loc[raw_df['Varietal Group'] == 'PINOT BLANC']
pinot_grigio = raw_df.loc[raw_df['Varietal Group'] == 'PINOT GRIGIO']
shiraz = raw_df.loc[raw_df['Varietal Group'] == 'SHIRAZ']
zinfandel = raw_df.loc[raw_df['Varietal Group'] == 'ZINFANDEL']
white_blend = raw_df.loc[raw_df['Varietal Group'] == 'WHITE BLEND']
sav_blanc = raw_df.loc[raw_df['Varietal Group'] == 'SAUVIGNON BLANC']
viognier = raw_df.loc[raw_df['Varietal Group'] == 'VIOGNIER']
rose = raw_df.loc[raw_df['Varietal Group'] == 'ROSE']
gewurz = raw_df.loc[raw_df['Varietal Group'] == 'GEWURZTRAMINER']
tempranillo = raw_df.loc[raw_df['Varietal Group'] == 'TEMPRANILLO']
malbec = raw_df.loc[raw_df['Varietal Group'] == 'MALBEC']
unassigned = raw_df.loc[raw_df['Varietal Group'] == 'UNASSIGNED']
other = raw_df.loc[raw_df['Varietal Group'] == 'OHER']

In [71]:
cabernet_franc.head()

,BC Sales Channel,AWC and Comp,Date,Brand,Varietal Group,9L Cases,AWC FYr & Pd No,Avg Sales Price,Sales Dollars
0,BCLS,CONSTELLATION BRANDS,2017-02,BLACK SAGE VINEYARD,CABERNET FRANC,17,201712,$28.74,"5,912"
8,BCLS,CONSTELLATION BRANDS,2017-02,INNISKILLIN ICEWINE,CABERNET FRANC,3,201712,$69.99,"5,663"
12,BCLS,CONSTELLATION BRANDS,2017-02,INNISKILLIN NIAGARA,CABERNET FRANC,2,201712,$17.24,465
25,BCLS,CONSTELLATION BRANDS,2017-02,JACKSON TRIGGS NIAGARA,CABERNET FRANC,0,201712,$44.72,368
77,BCLS,CONSTELLATION BRANDS,2018-03,BLACK SAGE VINEYARD,CABERNET FRANC,99,201801,$28.74,"34,285"


In [98]:
varietal = cabernet_franc
cols = ['BC Sales Channel',	'AWC and Comp', '9L Cases', 'AWC FYr & Pd No',	'Avg Sales Price']
varietal.drop(cols, axis=1, inplace=True)
varietal = varietal.sort_values('Date')
varietal.isnull().sum()

Date              0
Brand             0
Varietal Group    0
Sales Dollars     0
dtype: int64

In [103]:
varietal = varietal.groupby(['Date','Brand'])['Sales Dollars'].sum().reset_index()
varietal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           168 non-null    object
 1   Brand          168 non-null    object
 2   Sales Dollars  168 non-null    object
dtypes: object(3)
memory usage: 4.1+ KB


In [105]:
varietal = varietal.set_index('Date')
varietal.index

Index(['01/01/18', '01/01/18', '01/01/18', '01/01/18', '01/01/19', '01/01/19',
       '01/01/19', '01/01/19', '01/01/20', '01/01/20',
       ...
       '12/01/18', '12/01/18', '12/01/18', '12/01/19', '12/01/19', '12/01/19',
       '12/01/19', '12/01/20', '12/01/20', '12/01/20'],
      dtype='object', name='Date', length=168)

In [106]:
varietal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 01/01/18 to 12/01/20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Brand          168 non-null    object
 1   Sales Dollars  168 non-null    object
dtypes: object(2)
memory usage: 3.9+ KB


In [107]:
y = varietal['Sales Dollars'].resample('MS').mean()



TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [108]:
y.plot(figsize=(15, 6))
plt.show()

TypeError: no numeric data to plot